# 第二次课后习题

学号：20179065 $ \qquad $ 班级序列号：170230 $ \qquad $ 姓名：董秩序



## 0. 名词解释
* Eigendecomposition
>特征分解（Eigendecomposition），又称谱分解（Spectral decomposition）,是将矩阵分解为由其特征值和特征向量表示的矩阵之积的方法。只有对可对角化矩阵才可以特征分解。
* Eigenvalues
>设 A 是n阶方阵，如果存在数m和非零n维列向量 x，使得 Ax=mx 成立，则称 m 是矩阵A的一个特征值（characteristic value)或本征值（eigenvalue)。
* Eigenvectors
>线性变换的特征向量（本征向量）是一个非简并的向量，其方向在该变换下不变。该向量在此变换下缩放的比例称为其特征值（本征值）。

## 1.用numpy生成一组随机数，如何构建非奇异的矩阵A，对称的A，对称且正定的非奇异A；

### 非奇异矩阵的定义
若n阶矩阵A的行列式不为零，即 |A|≠0，则称A为非奇异矩阵，否则称A为奇异矩阵。n 阶方阵 A 是非奇异方阵的充要条件是 A 为可逆矩阵，也即A的行列式不为零。 即矩阵A可逆与矩阵A非奇异是等价的概念。
### 对称矩阵的判断
A的逆等于A
### 正定矩阵（positive definite）
广义定义：设M是n阶方阵，如果对任何非零向量z，都有zTMz> 0，其中zT 表示z的转置，就称M为正定矩阵。半正定矩阵包括了正定矩阵。判断正定不半正定的前提是对称矩阵。
> 正定矩阵的判断:所有的特征值都大于0

In [1]:
import numpy as np

#### 1.创建一个随机数值的方阵

In [2]:
A = np.random.rand(5**2).reshape(5, 5) #随机数生成5*5方阵
A*=10

In [3]:
A = A.astype(int) #转换类型
A

array([[5, 2, 8, 6, 3],
       [3, 3, 9, 8, 7],
       [5, 1, 0, 0, 2],
       [8, 7, 3, 5, 9],
       [3, 0, 1, 4, 0]])

In [4]:
value = np.linalg.det(A) #求矩阵的行列式,非零为非奇异矩阵
value

-2478.000000000002

#### 2.保留其上三角部分

In [5]:
A2 = np.triu(A) #保留其上三角部分
A2

array([[5, 2, 8, 6, 3],
       [0, 3, 9, 8, 7],
       [0, 0, 0, 0, 2],
       [0, 0, 0, 5, 9],
       [0, 0, 0, 0, 0]])

#### 3.将上三角”拷贝”到下三角部分

In [6]:
A2 += A2.T - np.diag(A2.diagonal()) #上三角加下三角减去对角线元素np.diag(A2.diagonal())
A2

array([[5, 2, 8, 6, 3],
       [2, 3, 9, 8, 7],
       [8, 9, 0, 0, 2],
       [6, 8, 0, 5, 9],
       [3, 7, 2, 9, 0]])

#### 4.验证对称

In [7]:
A2.T == A2

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

#### 5.求A的特征值

In [8]:
t = np.linalg.eigvals(A2)
t

array([ 24.73524557,   5.69469906,   1.69191286, -12.11560517,
        -7.00625232])

#### 6.产生正定矩阵

In [9]:
A3 = np.dot(A2,A2.transpose())
A3

array([[138, 157,  64, 103,  99],
       [157, 207,  57, 139, 117],
       [ 64,  57, 149, 138,  87],
       [103, 139, 138, 206, 119],
       [ 99, 117,  87, 119, 143]])

In [10]:
t = np.linalg.eigvals(A3) #特征值均大于0，为正定矩阵
t

array([611.83237327, 146.7878886 ,   2.86256914,  32.42959739,
        49.08757161])

## 2. 对A进行高斯消去Gaussian Elimination，The LU Decomposition，LDM Factorization， LDL Decomposition，Cholesky Decomposition

### 高斯消去（高斯消去法）
用来为线性方程组求解，当用于一个矩阵时，高斯消元法会产生出一个“行梯阵式”，使其先转换为一个上三角矩阵，再通过回代法，由下向上计算出未知值，并逐层向上带入。

#### 1. 将方阵A改写为5*6方程组

In [11]:
B = np.random.rand(5).reshape(5,1) #随机数生成每行方程的值
B*=10
B = B.astype(int)
B

array([[4],
       [7],
       [5],
       [5],
       [2]])

#### 2. 执行高斯消去法

首先A的列数为方程组的未知数个数，
然后B是方程组中每一个方程左边的未知数参数，
最后A4是方程组

求解是思路是首先通过矩阵运算，将当前的矩阵转化为上三角矩阵，然后再通过使用回代法来对其进行求解，最后输出未知数所对应的解。

In [12]:
from numpy import dot

In [13]:
def gauss(A,B):
    a = A.copy()
    c = B.copy() #复制可变对象
    n = len(c)
    for i in range(0,n-1):
        for j in range(i+1,n):
            if a[j,i] != 0.0:
                lam = a[j,i]/a[i,i]
                a[j,(i+1):n] = a[j,(i+1):n] - lam*a[i,(i+1):n:n]
                c[j] = c[j] - lam*c[i]
    for k in range(n-1,-1,-1):
        c[k] = (c[k] - dot(a[k,(k+1):n],c[(k+1):n]))/a[k,k]
    return c
c = gauss(A,B)  #得到方程的解
c

array([[ 0],
       [ 2],
       [-3],
       [ 3],
       [ 1]])

In [16]:
c2 = gauss(A2,B)
c2

array([[0],
       [2],
       [0],
       [0],
       [0]])

### The LU Decomposition
LU分解(LU Decomposition)是矩阵分解的一种，可以将一个矩阵分解为一个单位下三角矩阵和一个上三角矩阵的乘积（有时是它们和一个置换矩阵的乘积）。LU分解主要应用在数值分析中，用来解线性方程、求反矩阵或计算行列式。

>$Ax= (LU)x = L(Ux) = b$。令$Ux = y$，则原线性方程组$Ax = b$可首先求解向量$y$使$Ly = b$，然后求解$ Ux = y$，从而达到求解线性方程组$Ax= b$的目的。

LU分解在本质上是高斯消元法的一种表达形式。实质上是将A通过初等行变换变成一个上三角矩阵，其变换矩阵就是一个单位下三角矩阵。这正是所谓的*杜尔里特算法（Doolittle algorithm）*：从下至上地对矩阵A做初等行变换，将对角线左下方的元素变成零，然后再证明这些行变换的效果等同于左乘一系列单位下三角矩阵，这一系列单位下三角矩阵的乘积的逆就是L矩阵，它也是一个单位下三角矩阵。这类算法的复杂度一般在(三分之二的n三次方) 左右。

In [17]:
import pandas as pd

In [18]:
def LU_decomposition(A):
    n=len(A[0])
    L = np.zeros([n,n])
    U = np.zeros([n, n])
    for i in range(n):
        L[i][i]=1
        if i==0:
            U[0][0] = A[0][0]
            for j in range(1,n):
                U[0][j]=A[0][j]
                L[j][0]=A[j][0]/U[0][0]
        else:
                for j in range(i, n):#U
                    temp=0
                    for k in range(0, i):
                        temp = temp+L[i][k] * U[k][j]
                    U[i][j]=A[i][j]-temp
                for j in range(i+1, n):#L
                    temp = 0
                    for k in range(0, i ):
                        temp = temp + L[j][k] * U[k][i]
                    L[j][i] = (A[j][i] - temp)/U[i][i]
    return L,U
 
if __name__ == '__main__': 
    L,U=LU_decomposition(A)
    print(L,'\n',U) #L是对角元素都为1的下三角矩阵，D是对角元素都为正数的对角矩阵

[[ 1.          0.          0.          0.          0.        ]
 [ 0.6         1.          0.          0.          0.        ]
 [ 1.         -0.55555556  1.          0.          0.        ]
 [ 1.6         2.11111111  3.29411765  1.          0.        ]
 [ 0.6        -0.66666667  0.17647059 -1.81981982  1.        ]] 
 [[  5.           2.           8.           6.           3.        ]
 [  0.           1.8          4.2          4.4          5.2       ]
 [  0.           0.          -5.66666667  -3.55555556   1.88888889]
 [  0.           0.           0.          -2.17647059 -13.        ]
 [  0.           0.           0.           0.         -22.32432432]]


或者使用scipy模块

In [19]:
import pprint
import scipy
import scipy.linalg   # SciPy Linear Algebra Library
P, L, U = scipy.linalg.lu(A)  

print ("A:")
pprint.pprint(A)

print ("P:")
pprint.pprint(P)

print ("L:")
pprint.pprint(L)

print ("U:")
pprint.pprint(U)

A:
array([[5, 2, 8, 6, 3],
       [3, 3, 9, 8, 7],
       [5, 1, 0, 0, 2],
       [8, 7, 3, 5, 9],
       [3, 0, 1, 4, 0]])
P:
array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])
L:
array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.625     ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.375     , -0.11111111,  1.        ,  0.        ,  0.        ],
       [ 0.375     ,  0.77777778,  0.17391304,  1.        ,  0.        ],
       [ 0.625     ,  0.7037037 ,  0.97101449, -0.15102041,  1.        ]])
U:
array([[ 8.        ,  7.        ,  3.        ,  5.        ,  9.        ],
       [ 0.        , -3.375     , -1.875     , -3.125     , -3.625     ],
       [ 0.        ,  0.        ,  7.66666667,  5.77777778,  3.22222222],
       [ 0.        ,  0.        ,  0.        ,  3.55072464, -1.11594203],
       [ 0.        ,  0.        ,  0.        ,  0.    

### LDL Decomposition
若对称矩阵A 的各阶顺序主子式不为零时，A 可唯一分解为$A=LDL^T$

In [20]:
A2

array([[5, 2, 8, 6, 3],
       [2, 3, 9, 8, 7],
       [8, 9, 0, 0, 2],
       [6, 8, 0, 5, 9],
       [3, 7, 2, 9, 0]])

In [21]:
B

array([[4],
       [7],
       [5],
       [5],
       [2]])

In [22]:
def LDLTDCMP(n,A): #分解
    a = A.copy()
    for k in range(n):
        for m in range(k):
            a[k][k] -= a[m][k] * a[k][m]
        if a[k][k] == 0:
            print("\n\nERROR: LDL\' decompose failed !!\n")
            print("\n\nERROR: LDL\' decompose failed !!\n")
        for i in range(k+1,n):
            for m in range(k):
                a[k][i] = a[k][i] - a[m][i] * a[k][m]
                a[i][k] = a[k][i] / a[k][k] 
    return a
                
def LDLTBKSB(n,a,B): #解方程组
    b = B.copy()
    for i in range(n):
        for k in range(i):
            b[i] = b[i] - a[i][k] * b[k]
            
    for i in range(n-1,-1,-1):
        b[i] = b[i] / a[i][i]
        for k in range(i+1,n):
            b[i] = b[i] - a[k][i] * b[k]
    return b  

if __name__ == '__main__': 
    a = LDLTDCMP(len(B),A2)
    value = LDLTBKSB(len(B),a,B)
    print(value)

[[ 0]
 [-3]
 [ 0]
 [ 1]
 [ 0]]


### Cholesky Decomposition
Cholesky 分解是把一个对称正定的矩阵(A3)表示成一个下三角矩阵L和其转置的乘积的分解。它要求矩阵的所有特征值必须大于零，故分解的下三角的对角元也是大于零的。

Cholesky分解法又称平方根法，是当A为实对称正定矩阵时，LU三角分解法的变形。
通过直接比较A=L*L^T两边的对应元素来计算L，其中L^T为L的转置。

In [23]:
A3

array([[138, 157,  64, 103,  99],
       [157, 207,  57, 139, 117],
       [ 64,  57, 149, 138,  87],
       [103, 139, 138, 206, 119],
       [ 99, 117,  87, 119, 143]])

In [24]:
L = np.linalg.cholesky(A3)
L

array([[11.74734012,  0.        ,  0.        ,  0.        ,  0.        ],
       [13.36472753,  5.32766909,  0.        ,  0.        ,  0.        ],
       [ 5.4480418 , -2.96782588, 10.5124141 ,  0.        ,  0.        ],
       [ 8.76794227,  4.09538209,  9.73955783,  4.18234949,  0.        ],
       [ 8.42743965,  0.82016453,  4.13996839,  0.34151785,  7.35184455]])

In [25]:
np.dot(L, L.T.conj()) # 证明 L * L.H = A

array([[138., 157.,  64., 103.,  99.],
       [157., 207.,  57., 139., 117.],
       [ 64.,  57., 149., 138.,  87.],
       [103., 139., 138., 206., 119.],
       [ 99., 117.,  87., 119., 143.]])

## 5. Eigendecomposition：用NumPy求前面生成的不同矩阵A的特征值和特征向量，观察其特点

In [26]:
#调用numpy.linalg.eig（A）计算方阵的特征值和右特征向量。
#返回w:特征值
#v：归一化的（单位为“长度”）特征向量

In [27]:
A #随机方阵

array([[5, 2, 8, 6, 3],
       [3, 3, 9, 8, 7],
       [5, 1, 0, 0, 2],
       [8, 7, 3, 5, 9],
       [3, 0, 1, 4, 0]])

In [28]:
w1,v1 = np.linalg.eig(A)
w1,v1

(array([20.13826715+0.j        , -7.62742505+0.j        ,
         3.18440317+0.j        , -1.34762263+1.80277753j,
        -1.34762263-1.80277753j]),
 array([[ 0.45557868+0.j        ,  0.39772477+0.j        ,
          0.46199239+0.j        , -0.01391245-0.15868291j,
         -0.01391245+0.15868291j],
        [ 0.55080294+0.j        ,  0.53076418+0.j        ,
         -0.37468388+0.j        ,  0.70237384+0.j        ,
          0.70237384-0.j        ],
        [ 0.16080041+0.j        , -0.38579627+0.j        ,
          0.50389402+0.j        ,  0.02354654+0.22263328j,
          0.02354654-0.22263328j],
        [ 0.64905745+0.j        , -0.60537735+0.j        ,
         -0.60409364+0.j        ,  0.06335226-0.26710753j,
          0.06335226+0.26710753j],
        [ 0.20477265+0.j        ,  0.21162205+0.j        ,
         -0.16533816+0.j        , -0.53295085+0.26791903j,
         -0.53295085-0.26791903j]]))

In [29]:
A2 #随机对称方阵

array([[5, 2, 8, 6, 3],
       [2, 3, 9, 8, 7],
       [8, 9, 0, 0, 2],
       [6, 8, 0, 5, 9],
       [3, 7, 2, 9, 0]])

In [30]:
w2,v2 = np.linalg.eig(A2)
w2,v2

(array([ 24.73524557,   5.69469906,   1.69191286, -12.11560517,
         -7.00625232]),
 array([[ 0.41615972,  0.56326653,  0.60628271,  0.37484372,  0.03814932],
        [ 0.5095038 , -0.06584998, -0.60954173,  0.54551126, -0.25873925],
        [ 0.3533697 ,  0.56271935, -0.3672118 , -0.64899148,  0.04942004],
        [ 0.52137294, -0.48588442,  0.35494275, -0.374438  , -0.47528772],
        [ 0.41293714, -0.35448253,  0.00716179, -0.02271327,  0.83860472]]))

In [31]:
A3 #正定对称方阵

array([[138, 157,  64, 103,  99],
       [157, 207,  57, 139, 117],
       [ 64,  57, 149, 138,  87],
       [103, 139, 138, 206, 119],
       [ 99, 117,  87, 119, 143]])

In [32]:
w3,v3 = np.linalg.eig(A3)
w3,v3

(array([611.83237327, 146.7878886 ,   2.86256914,  32.42959739,
         49.08757161]),
 array([[-0.41615972, -0.37484372, -0.60628271,  0.56326653,  0.03814932],
        [-0.5095038 , -0.54551126,  0.60954173, -0.06584998, -0.25873925],
        [-0.3533697 ,  0.64899148,  0.3672118 ,  0.56271935,  0.04942004],
        [-0.52137294,  0.374438  , -0.35494275, -0.48588442, -0.47528772],
        [-0.41293714,  0.02271327, -0.00716179, -0.35448253,  0.83860472]]))

对于对称矩阵来说，主元的符号与特征值得符号是相同的。
即正主元的个数等于正的特征值的个数。

正定矩阵的所有特征值都是正数。所有的主元都是正数。

# 实验
1. 用第一节随机数据矩阵，进行 Eigendecomposition，观察特征值，特征向量特点；
2. 求上述矩阵的范数；
3. 计算上述矩阵的Covariance Matrices并进行特征值分解

### 1. Eigendecomposition
如果矩阵对某一个向量或某些向量只发生伸缩变换，不对这些向量产生旋转的效果，那么这些向量就称为这个矩阵的特征向量，伸缩的比例就是特征值。

物理的含义就是运动的图景：特征向量在一个矩阵的作用下作伸缩运动，伸缩的幅度由特征值确定。特征值大于1，所有属于此特征值的特征向量变长；特征值大于0小于1，特征向量变短；特征值小于0，特征向量缩过了界，反方向到0点那边。

将矩阵分解为**由其特征值和特征向量表示的矩阵之积**的方法。
方阵Ａ的特征向量是指与Ａ相乘后相当于对该向量进行缩放的非零向量v，标量λ被称为这个特征向量对应的特征值。如果**v**是A的特征向量，那么任何缩放后的向量也是A的特征向量。
需要注意只有对可对角化矩阵才可以施以特征分解。

步骤：
1. 计算实矩阵的**特征多项式**
$f(λ)=λE-A$
2. 令 $f(λ)=0$，建立齐次线性方程组，求出特征多项式 $f(λ)$的全部根，即A的全部特征值
3. 求出A 的全部特征向量

所有特征值都是正数的矩阵被称为正定；所有特征值都是非负数的矩阵被称为半正定。同样的，所有特征值都是负数的矩阵被称为负定；所有特征值都是非正数的矩阵被称为半负定。

代码操作见 5.

### 2. 范数
1-范数：列和范数，即所有矩阵列向量绝对值之和的最大值

2-范数：谱范数，即A'A矩阵的最大特征值的开平方

∞-范数：行和范数，即所有矩阵行向量绝对值之和的最大值

F-范数：Frobenius范数，即矩阵元素绝对值的平方和再开平方

In [33]:
print('默认的矩阵范数(F范数):', 
      '\nA:',np.linalg.norm(A),
      '\nA2:',np.linalg.norm(A2),
      '\nA3:',np.linalg.norm(A3))

默认的矩阵范数(F范数): 
A: 25.179356624028344 
A2: 29.03446228191595 
A3: 631.9454090346728


In [34]:
print('矩阵2范数:', 
      '\nA:',np.linalg.norm(A,ord=2),
      '\nA2:',np.linalg.norm(A2,ord=2),
      '\nA3:',np.linalg.norm(A3,ord=2))

矩阵2范数: 
A: 23.199104375929664 
A2: 24.735245567298623 
A3: 611.832373274566


### 3. Covariance Matrices
numpy.cov()可用于计算协方差矩阵

In [35]:
np.cov(A)

array([[ 5.7 ,  4.5 , -1.35, -4.65,  2.15],
       [ 4.5 ,  8.  , -4.  , -4.25,  0.5 ],
       [-1.35, -4.  ,  4.3 ,  3.45,  0.55],
       [-4.65, -4.25,  3.45,  5.8 , -1.05],
       [ 2.15,  0.5 ,  0.55, -1.05,  3.3 ]])

In [36]:
w,v = np.linalg.eig(np.cov(A))
w,v

(array([1.77370980e+01, 5.29370891e+00, 1.21132999e-15, 1.54580459e+00,
        2.52338853e+00]),
 array([[-0.48417563, -0.48507371,  0.58274324,  0.43479401, -0.04052088],
        [-0.60043208,  0.27297529, -0.29064136, -0.03904304, -0.69207589],
        [ 0.35488512, -0.52377495, -0.51848082,  0.47559738, -0.32357553],
        [ 0.51522578,  0.07786983,  0.54915152, -0.13497563, -0.63929092],
        [-0.11685132, -0.64014379, -0.07448598, -0.75167714, -0.07742748]]))

In [37]:
np.cov(A2)

array([[  5.7 ,   4.45,  -6.8 ,  -7.85,  -0.45],
       [  4.45,   9.7 , -13.3 ,  -5.85,  -1.2 ],
       [ -6.8 , -13.3 ,  19.2 ,   7.9 ,   1.8 ],
       [ -7.85,  -5.85,   7.9 ,  12.3 ,   0.35],
       [ -0.45,  -1.2 ,   1.8 ,   0.35,  13.7 ]])

In [38]:
w,v = np.linalg.eig(np.cov(A2))
w,v

(array([3.71013381e+01, 1.36341027e+01, 9.14726405e+00, 1.14809133e-15,
        7.17295087e-01]),
 array([[ 3.25898219e-01,  9.09836039e-02, -3.85155436e-01,
         -5.77350269e-01,  6.35479576e-01],
        [ 4.80814447e-01, -4.77585636e-03,  3.10006399e-01,
         -5.77350269e-01, -5.82543868e-01],
        [-6.82071047e-01,  1.55891532e-02, -4.49855300e-01,
         -4.61880215e-01, -3.44721541e-01],
        [-4.35093045e-01, -1.64465117e-01,  7.25055460e-01,
         -3.46410162e-01,  3.71402540e-01],
        [-8.98940896e-02,  9.82042453e-01,  1.65759198e-01,
         -2.19248779e-16,  5.96339917e-03]]))

In [39]:
np.cov(A3)

array([[ 1313.7 ,  1938.4 , -1348.75,  -308.25,   155.  ],
       [ 1938.4 ,  3022.8 , -1830.5 ,   -22.5 ,   373.  ],
       [-1348.75, -1830.5 ,  1788.5 ,  1015.75,  -276.  ],
       [ -308.25,   -22.5 ,  1015.75,  1541.5 ,    83.  ],
       [  155.  ,   373.  ,  -276.  ,    83.  ,   456.  ]])

In [40]:
w,v = np.linalg.eig(np.cov(A3))
w,v

(array([ 5.78315600e+03,  1.90053991e+03,  4.36523834e+02,  2.28025718e+00,
        -7.32231196e-14]),
 array([[-0.47000039, -0.08078759, -0.22683825, -0.76457608,  0.36951431],
        [-0.68860683, -0.3810809 , -0.09300432,  0.58091269,  0.18571115],
        [ 0.52111991, -0.32774426, -0.17781883,  0.13251021,  0.75620053],
        [ 0.16091099, -0.8556118 ,  0.02672748, -0.22698865, -0.435658  ],
        [-0.08638266, -0.09361016,  0.95265981, -0.09423938,  0.25948681]]))